In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import re
import os
pd.set_option('max_colwidth', 400)

# Préprocessing du texte

accents, contractions, lowercase, newlines, lemmatization, stemming

In [2]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [3]:
def text_preprocessing(serie):
    
    # lowercase
    serie = serie.map(lambda x: x.lower())   

    # remove extra newlines
    serie = serie.map(lambda x: re.sub(r'[\r|\n|\r\n]+', ' ', x))

    # remove @tag
    serie = serie.map(lambda x: re.sub(r'@[A-Za-z0-9]+','', x))

    # remove URL
    serie = serie.map(lambda x: re.sub('https?://[A-Za-z0-9./]+','', x))

    # remove hashtag and numbers
    serie = serie.map(lambda x: re.sub("[^a-zA-Z]", " ", x))

    # tokenization
    serie = serie.map(word_tokenize)

    # remove stop words
    stop_words = set(stopwords.words('english'))
    serie = serie.map(lambda x: [word for word in x if word not in stop_words])

    # lemmatization    
    serie = serie.map(nltk.tag.pos_tag)
    serie = serie.map(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

    wordnet_lemmatizer = WordNetLemmatizer()
    serie = serie.map(lambda x: [wordnet_lemmatizer.lemmatize(word, tag) for (word, tag) in x])

    serie = serie.map(lambda x: ' '.join(word for word in x))

    return serie

# Sentiment140
## Téléchargement de la base Sentiment140

In [2]:
import requests
import tempfile
import zipfile

url, destname = 'http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip', 'sentiment140'
temporary_location = "temp"

def download_unzip(url, dirname = tempfile.gettempdir(), destname = "file"):
    myfile = requests.get(url)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    open(os.path.join(dirname, destname + '.zip'), 'wb').write(myfile.content)
    with zipfile.ZipFile(os.path.join(dirname, destname + '.zip'), 'r') as zip_ref:
        zip_ref.extractall(os.path.join(dirname, destname))
        
download_unzip(url, dirname=temporary_location, destname=destname)

In [9]:
trainfile = os.path.join(temporary_location, destname, "training.1600000.processed.noemoticon.csv")
# testfile = os.path.join(temporary_location, destname, "testdata.manual.2009.06.14.csv")
columns = ['sentiment','id','date','query_string','user','text']

In [10]:
# Import train data set
df = pd.read_csv(trainfile,
                 header=None, 
                 names=columns, 
                 encoding='latin-1')
df = df.sample(100000)
df

,sentiment,id,date,query_string,user,text
1325893,4,2015186894,Wed Jun 03 03:57:47 PDT 2009,NO_QUERY,Neta_Lifshitz,"Funny Video, worth a listen if you got a minut..."
346430,0,2016155708,Wed Jun 03 06:16:20 PDT 2009,NO_QUERY,beatty_2002,"I'm beginning to slow down now, tired"
863698,4,1677152268,Sat May 02 00:41:15 PDT 2009,NO_QUERY,Henwii,thought I had lost this tshirt (which I love) ...
734981,0,2264677911,Sun Jun 21 04:37:03 PDT 2009,NO_QUERY,DiscoverNicole,"Still buzzing from last night, but now im have..."
1265463,4,1999454361,Mon Jun 01 20:14:46 PDT 2009,NO_QUERY,crisbeee,@BlueLightTech welcome back to the twittervers...
...,...,...,...,...,...,...
758314,0,2295685410,Tue Jun 23 08:14:45 PDT 2009,NO_QUERY,polarna10,@shakirayshakira I hate you color me green wi...
936279,4,1792961448,Thu May 14 00:51:23 PDT 2009,NO_QUERY,Crichton_Kicks,"@_Flik_ Ah, yeah I suppose, as long as you don..."
1173397,4,1980758885,Sun May 31 07:36:01 PDT 2009,NO_QUERY,serenawet,@DreamNetJade Had a blast last night with you ...
883116,4,1686075988,Sun May 03 03:37:46 PDT 2009,NO_QUERY,MizSadittyFancy,@Jawannsample ummmm sum egg whites and turkey ...


## Préprocessing Sentiment140

In [20]:
df[['text']].head(10)

,text
1325893,"Funny Video, worth a listen if you got a minute to spare http://bit.ly/19Jf7Z"
346430,"I'm beginning to slow down now, tired"
863698,thought I had lost this tshirt (which I love) ~2 yrs ago. Tony just found &amp; returned it @ West Covina http://loopt.us/M0_NCg.t
734981,"Still buzzing from last night, but now im have post gig depression"
1265463,@BlueLightTech welcome back to the twitterverse. How r u doing darling? Hope u have a wonderful day! Take Care!
1060723,@fleckman youre outtahere pal!
1246418,"@gyratory, @korruptor: congratulations on the Crackdown 2 announcement! Here's hoping Realtime Worlds aren't gonna come slash yr tires."
412698,Man I am really trying to get into Harper's Island but I just can't. That NHL standley cup game on NBC is a blow out right now...
443846,"@Julesxv you might be able to download the drivers from the brand website, they may not be fully supportive of Linux though"
469672,@cocoward ARE YOU SHITTING ME??? who who who how how did you find out??? AND NO im not lying slutttt you look beautiful


In [21]:
%%time
df.text = text_preprocessing(df.text)

Wall time: 1min 32s


In [22]:
df[['text']].head(10)

,text
1325893,funny video worth listen get minute spare
346430,begin slow tire
863698,thought lose tshirt love yr ago tony find amp return west covina ncg
734981,still buzz last night im post gig depression
1265463,welcome back twitterverse r u darling hope u wonderful day take care
1060723,youre outtahere pal
1246418,congratulation crackdown announcement hop realtime world gon na come slash yr tire
412698,man really try get harper island nhl standley cup game nbc blow right
443846,might able download driver brand website may fully supportive linux though
469672,shit find im lie slutttt look beautiful


In [23]:
%%time
# Sauvegarde
df.to_pickle(os.path.join("data", "sentiment140", "train.bz2"))

Wall time: 884 ms


# Base Twitter webscrappée
## Chargement de la base

In [4]:
df = pd.read_pickle(os.path.join("data", "web", "web_parse.bz2"))
df

,search,author,time,url,text,lang,reply
0,biden,Christopher Suprun #WeArePerseus,2019-01-01 23:59:59,/TheChrisSuprun/status/1080252328886915072,Off top of my head...Forgive spelling\n\nHarris (CA)\nORourke (TX)\nCastro (TX)\nMoulton (MA)\nWarren (MA)\nBiden (DE)\nSanders (VT)\nCuomo (NY)\nGillibrand (NY)\nKlobuchar (MN)\nBrown (OH)\nGabbard (HA)\nMcAuliffe (VA)\nLandrieu (LA)\nBooker (NJ) \nBloomberg (NY)\nSchultz (WA)\nGarcetti (CA),en,False
1,biden,robbiewithoutanynumbers,2019-01-01 23:59:54,/robbienotrobin/status/1080252304509599744,"I kinda like Kamala/Beto, with Biden as Chif Of Staff. Just sayin...",en,False
2,biden,Linda Hirshman 🏊🏻‍♂️,2019-01-01 23:59:50,/LindaHirshman1/status/1080252291184189441,also Biden to Grassley. Did you know he wrote an entire memoir story of his whole life without once mentioning Anita Hill. Do we think he forgot her?,en,False
3,biden,Carol Price,2019-01-01 23:59:43,/CarolP1941/status/1080252261547180032,I am anxious to see who can pull us out of this mess Trump has got us into. It will take a strong willed person with a VP that works beside him like Biden did with Obama.,en,False
4,biden,Oliver Anderson,2019-01-01 23:59:34,/bogart7777/status/1080252220204027904,"Black people, please don’t trust Joe Biden, he wrote the crime bill(1994) that sent thousands of Black people to prison. He is not good for the Black community.",en,False
...,...,...,...,...,...,...,...
2901,trump,RobDogDiggity,2019-12-31 23:59:06,/robdogdiggity/status/1212161285648977920,"Oh, if he would’ve pulled out a gun and shot one of them, he’d be there. They’d have marches, microphones in their faces, and blaming President Trump for it.",en,False
2902,trump,the other red [Drapeau des États-Unis][Drapeau de la Macédoine],2019-12-31 23:59:05,/zivvy_1/status/1212161285040869383,"So no reality? THAT is how we get another 4 years. Stick Biden on a stage with Trump & he’ll only leave a carcass with Hunter, Biden’s voting record and mumbling gaffes. \n\nThis process is about weeding that out. Not a participation trophy. \n\nUnite behind the fittest-not just any1",en,False
2903,trump,PeterDavies,2019-12-31 23:59:05,/PeterMDavies80/status/1212161285036630016,"Trump strong! He's so tough! He's gonna show the world who's boss!\n\nwell, except for the psychopath who's building nuclear weapons and threatening the United States with them - i mean what are we supposed to go to war or something come on guys we gotta just do what he says",en,False
2904,trump,"No, Donny, you did not win the election.",2019-12-31 23:59:05,/Vote4USA2020/status/1212161284617179136,That is so true. I think the ‘election’ of someone a bigoted as trump opened Pandora’s [Paquet].,en,False


In [5]:
def filtre(text):
    lower_text = text.lower()
    if "trumps" in lower_text:
        if "trump" in lower_text.replace("trumps", ""):
            return True
        else:
            return False
    else:
        return True

La recherche du mot `trump` dans Twitter fournit aussi des tweets comprenants le mot `trumps` (issu du nom ou verbe `trump`), on supprime donc ces tweets.

In [6]:
df = df[(df.search == 'biden') | (df.search == "trump") & (df.text.apply(filtre))]
df.shape

(2868, 7)

In [7]:
df[['text']].head(10)

,text
0,Off top of my head...Forgive spelling\n\nHarris (CA)\nORourke (TX)\nCastro (TX)\nMoulton (MA)\nWarren (MA)\nBiden (DE)\nSanders (VT)\nCuomo (NY)\nGillibrand (NY)\nKlobuchar (MN)\nBrown (OH)\nGabbard (HA)\nMcAuliffe (VA)\nLandrieu (LA)\nBooker (NJ) \nBloomberg (NY)\nSchultz (WA)\nGarcetti (CA)
1,"I kinda like Kamala/Beto, with Biden as Chif Of Staff. Just sayin..."
2,also Biden to Grassley. Did you know he wrote an entire memoir story of his whole life without once mentioning Anita Hill. Do we think he forgot her?
3,I am anxious to see who can pull us out of this mess Trump has got us into. It will take a strong willed person with a VP that works beside him like Biden did with Obama.
4,"Black people, please don’t trust Joe Biden, he wrote the crime bill(1994) that sent thousands of Black people to prison. He is not good for the Black community."
5,Question is whether there are more of these guys or never-Biden youngsters in the Dem primary/convincable electorate. The oldsters the FYI vote in heavier numbers...
6,"What % of @nytimes profits is driven by spending from Russia, China, and Saudi Arabia? Must be significant firvthen to attack lifelong public servants like Joe Biden. Why don't you apply resources to investigating dark money influence instead of hit pieces on good men?"
7,"I REALLY wish Democrats would stop trying to push for Biden as a presidential candidate just because they can't think of anybody more visible.\n\nSeriously, Biden is an AWFUL choice for multiple reasons."
8,Biden/Obama ticket!!! @JoeBiden
9,I will. But then I voted Hillary. My Dad (and lots of other Republicans) didn't vote for either. He would vote Biden (Obama's VP) however.


In [8]:
%%time
df.text = text_preprocessing(df.text)

Wall time: 5.1 s


C:\Users\pasqu\anaconda3\envs\data-2A\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
df[['text']].head(10)

,text
0,top head forgive spelling harris ca orourke tx castro tx moulton warren biden de sander vt cuomo ny gillibrand ny klobuchar mn brown oh gabbard ha mcauliffe va landrieu la booker nj bloomberg ny schultz wa garcetti ca
1,kinda like kamala beto biden chif staff sayin
2,also biden grassley know write entire memoir story whole life without mention anita hill think forgot
3,anxious see pull u mess trump get u take strong will person vp work beside like biden obama
4,black people please trust joe biden write crime bill send thousand black people prison good black community
5,question whether guy never biden youngster dem primary convincable electorate oldster fyi vote heavy number
6,profit drive spending russia china saudi arabia must significant firvthen attack lifelong public servant like joe biden apply resource investigate dark money influence instead hit piece good men
7,really wish democrat would stop try push biden presidential candidate think anybody visible seriously biden awful choice multiple reason
8,biden obama ticket
9,vote hillary dad lot republicans vote either would vote biden obama vp however


In [11]:
%%time
# Sauvegarde
df.to_pickle(os.path.join("data", "web", "web.bz2"))

Wall time: 113 ms
